## Tour of ROS tools with examples

ROS has plenty of tools that can help prototype, test and deploy robots. 

We will focus of some of the most important:

 - **[Rviz](http://wiki.ros.org/rviz)**: which is a tool to visialize the state of the robot in 3D as well as to see its sensor readouts

 - **[Tf](http://wiki.ros.org/tf2)**: which is used to simplify calculations of robot's kinematics

 - **[Rosbag](http://wiki.ros.org/rosbag)**: which is used to log Robot experiments



Here we will use the local server:

In [ ]:
%env ROS_MASTER_URI=http://localhost:11311

### Rviz

**[Rviz](http://wiki.ros.org/rviz)** is a visualization tool for ROS which means that it is very good in understanding standard ROS messages and parameters, and translating them into understandable visualizations. 

Normally you don't program the Rviz itself but send information to appropriate topics and parameters. You can however, write additional toolboxes for it. 



In [ ]:
# we launch the rviz using the command rosrun rviz rviz
import roslaunch
import rospy
import rosparam

In [ ]:
rospy.init_node("tourist")

In [ ]:
### we will put robot description of Pi Robot (mymodelrobot.appspot.com) to the parameter server

with open("../../description/pi_robot.urdf", "r") as pi_robot_file:
    urdf_text= pi_robot_file.read()


rosparam.set_param_raw("robot_description",urdf_text)

In [ ]:
### we can now view the robot in rviz

launch = roslaunch.scriptapi.ROSLaunch()
launch.start()

node_rviz = roslaunch.core.Node("rviz",
                            "rviz", name="rviz") # this starts a ROS Node from ipython_robot_prototyping 
node_state_publisher= roslaunch.core.Node("robot_state_publisher", "robot_state_publisher", name="robot_state_publisher") #this starts our turtle

node_joint_publisher = roslaunch.core.Node("joint_state_publisher","joint_state_publisher", name="joint_state_publisher")

nodes=[node_rviz,node_state_publisher,node_joint_publisher]

In [ ]:
### this will start rviz and two helper programs, robot_state publisher and robot_joint_publisher
all_procesess=[launch.launch(node) for node in nodes]
    

The code above started 3 programs:

 1. rviz which is the visualizer
 2. robot_state_publisher which takes joint_state messages and using robot design from URDF from "robot_description" describes where each robot part should be ( forward kinematics)
 3. joint_state_publisher which is a helper node that just publishes that our joints are in 0 position
 
To see our robot in Rviz, you need to click Add and then select "Robot Model"

![](images/add_robot_rviz.png)

Also change the "fixed frame" to base_link 
You should see a Pi robot model in Rviz.

Normally we would get the state of the robot -- its joint positions/velocities/ accelerations from its sensors of from a simulation. We can still play around by publishing additional joint state messages.

In [ ]:
all_procesess[2].stop() #first we need to stop joint_state_publisher

In [ ]:
from sensor_msgs.msg import JointState
from std_msgs.msg import Header

pub = rospy.Publisher('joint_states', JointState, queue_size=10)

joint_names= ['torso_joint', 
'head_pan_joint', 'head_tilt_joint', 'left_shoulder_forward_joint',
'right_shoulder_forward_joint', 'left_shoulder_up_joint', 
'right_shoulder_up_joint', 'left_elbow_joint', 
'right_elbow_joint', 'left_wrist_joint', 'right_wrist_joint']
    
    

In [ ]:
joint_msg = JointState()
joint_msg.header = Header()
joint_msg.header.stamp = rospy.Time.now()
joint_msg.name = joint_names
joint_msg.position = [0.5,0,0,0,0,0,0,0,0,0,0]
joint_msg.velocity = []
joint_msg.effort = []

pub.publish(joint_msg)

This sends new joint position to our robot simulation, moving the torso.

We can send such position in a loop.

In [ ]:
def move_torso(angle):
    joint_msg = JointState()
    joint_msg.header = Header()
    joint_msg.header.stamp = rospy.Time.now()
    joint_msg.name = joint_names
    joint_msg.position = [angle,0,0,0,0,0,0,0,0,0,0]
    joint_msg.velocity = []
    joint_msg.effort = []

    pub.publish(joint_msg)

In [ ]:
move_torso(0)

In [ ]:
### we can create a small loop that moves shoulder 
rate = rospy.Rate(10) # 10Hz move 10 times per s

import math

### amplitude - how far torso will go
### omega - how fast torso will go

amplitude= math.pi/2
omega= 1.5
while not rospy.is_shutdown():
        t=rospy.Time.now().to_sec()
        move_torso(amplitude*math.sin(omega*t))
        rate.sleep()

### Exercise:

Try to also move a shoulder 

In [ ]:
# your code

We can add additional elements to Rviz scene, such as markers

In [ ]:
from visualization_msgs.msg import Marker

In [ ]:
vis_pub = rospy.Publisher("visualization_marker",Marker, queue_size=5 );

In [ ]:
marker = Marker()
marker.header.frame_id = "base_link";
marker.header.stamp = rospy.Time.now()
marker.ns = "markers";
marker.id = 0;
marker.type = 2 #sphere
marker.action =0 # add
marker.pose.position.x = 0.5;
marker.pose.position.y = 0.5;
marker.pose.position.z =0.5;
marker.pose.orientation.x = 0.0;
marker.pose.orientation.y = 0.0;
marker.pose.orientation.z = 0.0;
marker.pose.orientation.w = 1.0;
marker.scale.x = 0.1;
marker.scale.y = 0.1;
marker.scale.z = 0.1;
marker.color.a = 1.0; # Don't forget to set the alpha!
marker.color.r = 0.0;
marker.color.g = 1.0;
marker.color.b = 0.0;

vis_pub.publish( marker );

To see the result in Rviz you need to add new "Marker" display type. We can orbit around the robot similarely how you rotated the shoulder.

### Exercise:

Orbit the sphere around the robot. Create a function for x y similar to that:

        x = 0.5 sin (t* omega)

        y = 0.5 cos (t * omega)

### Other types of information

You can also see other types of information like 3d point clouds or camera streams. We will use usb_camera package to stream our images to ROS.


In [ ]:
camera_node=roslaunch.core.Node("cv_camera", "cv_camera_node", name="cv_camera") #this starts our turtle
nodes.append(camera_node)
all_procesess.append(launch.launch(camera_node))

This time you need to add "Image" and select topic as "usb_camera/image_raw"